In [58]:
!pip install plotly
!pip install seaborn

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Matrix

| Matrix | Math | Surface |
| --- | --- | --- |
| Positive Definite | positive eigenvalues | bowl |
| Negative Definite | negative eigenvalues | bowl |
| Singular | 0 is an eigenvalue | many local minima |
| Other | other | saddle points |

In [7]:
# W = np.array([[-8, 3], [3, -9]])  # negative-definite
W = np.array([[3, -3], [-3, 9]])  # positive-definite
# W = np.array([[1, 3], [3, 9]])  # singular
# W = np.array([[1, 8], [3, 3]])    # saddle points

lmbs, vecs = np.linalg.eig(W)
lmb1, lmb2 = lmbs[0], lmbs[1]
vec1, vec2 = vecs[:, 0], vecs[:, 1]
print(lmb1, vec1)
print(lmb2, vec2)
print(W @ vec1 - lmb1 * vec1)


p = 50
x1s = np.linspace(-10, 10, p)
x2s = np.linspace(-10, 10, p)
# x is [p, p, 2]
x = np.stack(np.meshgrid(x1s, x2s), axis=-1)

y = np.zeros((p, p))
for i in range(p):
    for j in range(p):
        y[i][j] = x[i][j].reshape(1, 2) @ W @ x[i][j]

fig = make_subplots(rows=1, cols=2,specs=[[{'type': 'surface'}, {'type': 'heatmap'}]])
        
fig.add_trace(
    go.Surface(
        x=x[..., 0], 
        y=x[..., 1], 
        z=y
    ),
    row=1, col=1
)        
fig.add_trace(
    go.Heatmap(z=y, x=x1s, y=x2s),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(x=(0, vec1[0]), y=(0, vec1[1])),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=(0, vec2[0]), y=(0, vec2[1])),
    row=1, col=2
)

fig

1.7573593128807152 [-0.92387953 -0.38268343]
10.242640687119286 [ 0.38268343 -0.92387953]
[ 8.88178420e-16 -3.33066907e-16]


In [33]:
def is_pos_def(x):
    """check if a matrix is symmetric positive definite"""
    return np.all(np.linalg.eigvals(x) > 0)

def steepest_descent_store_result(A, b, x):
    x_steps = [x]
    y_steps = [0.5 * x @ A @ x - x @ b]
    
    if (is_pos_def(A) == False) | (A != A.T).any():
        raise ValueError('Matrix A needs to be symmetric positive definite (SPD)')
    r = b - A @ x
    k = 0
    while np.linalg.norm(r) > 1e-10 :
        p = r
        q = A @ p
        alpha = (p @ r) / (p @ q)
        x = x + alpha * p
        r = r - alpha * q
        k =+ 1
        x_steps.append(x)
        y_steps.append(0.5 * x @ A @ x - x @ b)

    return x, x_steps, y_steps

def conjugate_gradient_store_result(A, b, x0):
    if (is_pos_def(A) == False) | (A != A.T).any():
        raise ValueError('Matrix A needs to be symmetric positive definite (SPD)')
    k = 0
    x = x0
    r = b - A @ x0
    x_steps = [x]
    y_steps = [0.5 * x @ A @ x - x @ b]
    while np.linalg.norm(r) > 1e-10 :
        if k == 0:
            p = r
            # p = np.array([1.5, 1.5])
        else: 
            gamma = - (p @ A @ r)/(p @ A @ p)
            p = r + gamma * p
        alpha = (p @ r) / (p @ A @ p)
        x = x + alpha * p
        r = r - alpha * (A @ p)
        k =+ 1
        x_steps.append(x)
        y_steps.append(0.5 * x @ A @ x - x @ b)
        print(np.linalg.norm(r))

    return x, x_steps, y_steps

def viz_descent(x_steps, y_steps):
    size = 50
    x1s = np.linspace(-6, 6, size)
    x2s = np.linspace(-6, 6, size)
    x1, x2  = np.meshgrid(x1s, x2s)
    Z = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            x = np.array([x1[i,j], x2[i,j]])
            Z[i,j] = 0.5 * x @ A @ x - x @ b
    
    fig = go.Figure()
    fig.add_trace(
        go.Surface(x=x1s, y=x2s, z=Z)
    )
    fig.add_trace(
        go.Scatter3d(
            x=np.asarray(x_steps)[:, 0], 
            y=np.asarray(x_steps)[:, 1], 
            z=np.asarray(y_steps),
            marker=dict(
                size=2,
                symbol='circle'
            )
        )
    )
    return fig
    
A = np.array([
    [3, 2],
    [2, 3]
])
b = np.array([-2, 7])
x0 = np.array([4, 4])

# visualize steepest descent method
# _, x_steps, y_steps = steepest_descent_store_result(A, b, x0)
# print(y_steps)
# viz_descent(x_steps, y_steps)

# visualize conjugate gradient method
_, x_steps, y_steps = conjugate_gradient_store_result(A, b, x0)
print(y_steps)
viz_descent(x_steps, y_steps)


5.188183934053369
2.8435583831733384e-15
[60.0, -8.709152433129228, -21.5]
